#### Korter Data Scrappinig

In [1]:
import pandas as pd
import numpy as np

# https://developer.apple.com/documentation/webkit/testing_with_webdriver_in_safari
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.support.ui import Select
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium import webdriver

from bs4 import BeautifulSoup

import requests
import unittest
import time
import datetime
import re
import datetime

In [3]:
x = datetime.datetime.now()

# CHANGE
# city = str(input("Type the name of the city: "))
# cities = ["Bucuresti", "Cluj", "Timisoara", "Iasi", "Constanta", "Craiova", "Brașov", "Galați", "Ploiești"]
# cities = ["Bucuresti", "Cluj", "Timisoara", "Iasi", "Constanta"]
cities = ["Craiova", "Brașov", "Galați", "Ploiești", "Oradea", "Brăila", "Arad", "Pitești", "Sibiu", "Bacău"]

column_names = ["name", "address", "developer", "city", "date", "href", "main_price_from"]
residential = pd.DataFrame(columns = column_names)

for city in cities:
    url = "https://korter.ro/ansambluri-rezidentiale-in-" + city
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    check = soup.find(class_="ListingHeader__StyledComment-sc-6vo33j-2")
    # print(check.prettify())
    # [type(item) for item in list(check.children)]
    number = list(check.children)[2]
    number = int(number) 
    number = number / 20
    number_pages = int(number) + 2
    
    # for pg in range(1, number):
    for pg in range(1, number_pages):
        page_num = str(pg)
        url = "https://korter.ro/ansambluri-rezidentiale-in-" + city + "?page=" + page_num
        # page = driver.get(url)
        # soup = BeautifulSoup(driver.page_source, 'html.parser')
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        # print(soup.prettify())

        buildinigs_stream = soup.find(class_="BuildingsStream__StyledCardsWrapper-sc-1umumy-1")
        
        building_name = []
        building_name_tags = buildinigs_stream.select(".buildingCard__StyledWrapper-sc-1t8cw05-2 .buildingCard__StyledBuildingName-sc-1t8cw05-5")
        building_name_e = [bnt.get_text() for bnt in building_name_tags]
        building_name.extend(building_name_e)

        address = []
        address_tags = buildinigs_stream.select(".buildingCard__StyledWrapper-sc-1t8cw05-2 .buildingCard__StyledBuildingLocation-sc-1t8cw05-6")
        address_e = [at.get_text() for at in address_tags]
        address.extend(address_e)
    
        price = []
        price_tags = buildinigs_stream.select(".buildingCard__StyledWrapper-sc-1t8cw05-2 .Price__StyledBasePrice-sc-1ayg5nr-2")
        price_e = [pt.get_text() for pt in price_tags]
        price.extend(price_e)

        developer = []
        developer_tags = buildinigs_stream.select(".buildingCard__StyledWrapper-sc-1t8cw05-2 .buildingCard__StyledDeveloperName-sc-1t8cw05-10")
        developer_e = [dt.get_text() for dt in developer_tags]
        developer.extend(developer_e)

        # website = []
        # website_tags = buildinigs_stream.select(".buildingCard__StyledWrapper-sc-1t8cw05-2 .buildingCard__StyledBuildingSiteBlock-sc-1t8cw05-12")
        # website_e = [wt.get_text() for wt in website_tags]
        # website.extend(website_e)
        
        # buildinigs_stream_b = soup.find(class_="buildingCard__StyledWrapper-sc-1t8cw05-2 iPKkAv")
        href = []
        href_tags = buildinigs_stream.find_all('a', class_="Link__StyledLink-sc-1qa6dyr-0 givVbJ buildingCard__StyledMainLink-sc-1t8cw05-0 fjVoPz")
        href_e = [ht.get('href') for ht in href_tags]
        href.extend(href_e)   
        
        residential_city = pd.DataFrame({
            "name": building_name,
            "address": address,
            "developer": developer,
            "city": city,
            "date": str(x.strftime("%x")),
            "href": href,
            "main_price_from": price,
        })

        # residential_city['website'] = residential_city['website'].replace("pagina obiect", np.nan)
        residential = residential.append(residential_city, ignore_index = True)

residential

AttributeError: 'NoneType' object has no attribute 'children'

In [ ]:
# href2 is used in the blocks below as well
href2 = residential["href"].tolist()

# To check
# href2 = href2[12:30]

column_names = ["unedited", "type", "href"]
sketches_prices = pd.DataFrame(columns = column_names)

for resi in href2:
    url_resi = "https://korter.ro" + resi
    page_resi = requests.get(url_resi)
    soup_resi = BeautifulSoup(page_resi.content, 'html.parser')
    
    sket_pr_resi_type = []
    sket_pr_resi_type_tags = soup_resi.find_all(class_="UnitTypeRow__StyledUnitType-wfbjku-1 bjEpwX")
    sket_pr_resi_type_tags_e = [sprt.get_text() for sprt in sket_pr_resi_type_tags]
    sket_pr_resi_type.extend(sket_pr_resi_type_tags_e)
    sket_pr_resi_type

    sket_pr_resi_surface = []
    sket_pr_resi_surface_tags = soup_resi.find_all(class_="UnitTypeRow__StyledPriceArea-wfbjku-3 bJnUHx")
    sket_pr_resi_surface_tags_e = [sprt.get_text("|", strip=True) for sprt in sket_pr_resi_surface_tags]
    sket_pr_resi_surface.extend(sket_pr_resi_surface_tags_e)
    sket_pr_resi_surface
    
    sket_pr_resi_surface_df = pd.DataFrame({
        "unedited": sket_pr_resi_surface,
        "type": sket_pr_resi_type,
        "href": url_resi,
    })
    
    sketches_prices = sketches_prices.append(sket_pr_resi_surface_df, ignore_index = True)

sketches_prices.unedited = sketches_prices.unedited.str.replace('|', '')
sketches_prices.unedited = sketches_prices.unedited.str.replace('prețul se clarifică', 'de laprețul se clarifică')
sketches_prices.unedited = sketches_prices.unedited.str.replace('m2', '', 1)
sketches_prices[['area_from','price_m2_from','total_price_from']] = sketches_prices.unedited.apply( 
   lambda x: pd.Series(str(x).split("de la", 2)))
sketches_prices.price_m2_from = sketches_prices.price_m2_from.str.replace('pentru m2', '', 1)
sketches_prices.href = sketches_prices.href.str.replace('https://korter.ro', '', 1)
sketches_prices = sketches_prices.drop(['unedited'], axis=1)
sketches_prices.rename(columns={'href':'href2'}, inplace=True)
sketches_prices

In [ ]:
column_names = ["sketches_column", "href"]
sketches_all = pd.DataFrame(columns = column_names)

for resi in href2:
    url_resi = "https://korter.ro" + resi + "/schițele"
    page_resi = requests.get(url_resi)
    sketches_resi = BeautifulSoup(page_resi.content, 'html.parser')

    sketches = []
    sketches_tags = sketches_resi.find_all(class_="LayoutCard__StyledWrapper-sc-1j6xc9t-1 kvPUf")
    sketches_tags_e = [st.get_text("|", strip=True) for st in sketches_tags]
    sketches.extend(sketches_tags_e)
    sketches
    
    sketches_df = pd.DataFrame({
        "sketches_column": sketches,
        "href": url_resi,
    })
    
    sketches_all = sketches_all.append(sketches_df, ignore_index = True)
    
sketches_all[['del1','total_price','del2','total_space']] = sketches_all.sketches_column.apply(
    lambda x: pd.Series(str(x).split("|", 3)))
sketches_all.total_space = sketches_all.total_space.str.replace('|', '')
sketches_all.total_space = sketches_all.total_space.str.replace('m2', '')
sketches_all = sketches_all.drop(['sketches_column', 'del1', 'del2'], axis=1)
sketches_all.href = sketches_all.href.str.replace('https://korter.ro', '', 1)
sketches_all.href = sketches_all.href.str.replace('/schițele', '', 1)
sketches_all.rename(columns={'href':'href2'}, inplace=True)
sketches_all

In [ ]:
column_names = ["characteristic", "explanation", "href2"]
charactersitics_all = pd.DataFrame(columns = column_names)

for resi in href2:
    url_resi = "https://korter.ro" + resi
    page_resi = requests.get(url_resi)
    soup_resi = BeautifulSoup(page_resi.content, 'html.parser')
    table_resi = soup_resi.find(class_="PrimaryTableAttributesBlock__StyledAttributesTable-sc-1gdjixd-0 dRvDKU")
    web_resi = soup_resi.find(class_="Link__StyledLink-sc-1qa6dyr-0 fvupyZ")
    
    tds_resi_charactersitics = []
    tds_resi_charactersitics_tags = table_resi.find_all("td", class_="TableAttribute__StyledKey-xbfqge-1")
    tds_resi_charactersitics_tags_e = [tds.get_text() for tds in tds_resi_charactersitics_tags]
    tds_resi_charactersitics.extend(tds_resi_charactersitics_tags_e)
    tds_resi_charactersitics
    
    tds_resi_charactersitics_expl = []
    tds_resi_charactersitics_expl_tags = table_resi.find_all("td", class_=False)
    tds_resi_charactersitics_expl_tags_e = [tds.get_text() for tds in tds_resi_charactersitics_expl_tags]
    tds_resi_charactersitics_expl.extend(tds_resi_charactersitics_expl_tags_e)
    tds_resi_charactersitics_expl
    
    charactersitics = pd.DataFrame({
        "characteristic": tds_resi_charactersitics,
        "explanation": tds_resi_charactersitics_expl,
        "href2": url_resi,
    })
    
    charactersitics_all = charactersitics_all.append(charactersitics)

charactersitics_all.href2 = charactersitics_all.href2.str.replace('https://korter.ro', '', 1)
charactersitics_all

In [ ]:
href2 = residential["href"].tolist()

# To check
# href2 = href2[12:30]

column_names = ["website_link", "href2"]
website_all = pd.DataFrame(columns = column_names)

for resi in href2:   
    url_resi = "https://korter.ro" + resi
    page_resi = requests.get(url_resi)
    soup_resi = BeautifulSoup(page_resi.content, 'html.parser')
    external_links_resi = soup_resi.find_all(class_="ExternalLinks__StyledWrapper-sc-6pqnoc-1 eSjnUu")
    
    website_link = []
    website_e = [elr.get_text("|", strip=True) for elr in external_links_resi]
    website_link.extend(website_e)

    websites = pd.DataFrame({
        "website_link": website_link,
        "href2": url_resi,
    })
    
    website_all = website_all.append(websites)

website_all.href2 = website_all.href2.str.replace('https://korter.ro', '', 1)
website_all[['del1','website','del2']] = website_all.website_link.apply(
    lambda x: pd.Series(str(x).split("|", 2)))
website_all = website_all.drop(['website_link', 'del1', 'del2'], axis=1)
website_all

In [ ]:
# MERGING IT ALL

In [ ]:
charactersitics_allz = charactersitics_all.copy()
# https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html
charactersitics_allz = charactersitics_allz.pivot_table(index="href2", columns="characteristic", values="explanation", aggfunc='first')
charactersitics_allz.reset_index(inplace = True) 

# https://focaalvarez.medium.com/vlookup-and-index-match-equivalences-in-pandas-160ac2910399
residential.rename(columns={'href':'href2'}, inplace=True)
table_all_characteristics = residential.merge(charactersitics_allz,how='left',on='href2')
table_all_characteristics

In [ ]:
sketches_prices_2 = sketches_prices.pivot_table(index="href2", columns="type", values="price_m2_from", aggfunc='first')
sketches_prices_2.reset_index(inplace = True)

# sketches_prices_2 = sketches_prices.add_suffix('_price_m2_from')
# col_dict = {'type_price_m2_from': 'type', 'href2_price_m2_from': 'href2'} ## key→old name, value→new name
# sketches_prices_2.columns = [col_dict.get(x, x) for x in sketches_prices_2.columns]
sketches_prices_2

In [ ]:
table_all_char_sketpri = table_all_characteristics.merge(sketches_prices_2, how="left", on="href2")
table_all_char_sketpri

In [ ]:
# Use str.extract and pass a regex pattern to extract just the numeric parts
# sketches_all_2.total_price = sketches_all_2.total_price.str.extract('(\d+)', expand=False)
# sketches_all_2.total_space = sketches_all_2.total_space.str.encode("ascii", "ignore")
# sketches_all_2.total_space = sketches_all_2.total_space.astype(int)

sketches_all_2 = sketches_all.copy()
sketches_all_2.total_price = sketches_all_2.total_price.str.replace('€', '')
sketches_all_2.total_price = sketches_all_2.total_price.str.replace('[+TVA]', '')
sketches_all_2.total_price = sketches_all_2.total_price.str.replace(r"[a-zA-Z]",'')
sketches_all_2.total_price = sketches_all_2.total_price.str.replace("[ț ţ ă â]", '')
sketches_all_2.total_space = sketches_all_2.total_space.str.replace(r"[a-zA-Z]",'')
sketches_all_2.total_space = sketches_all_2.total_space.str.replace("[ț ţ â ă]", '')

# https://stackoverflow.com/questions/15891038/change-column-type-in-pandas
# df["a"] = pd.to_numeric(df["a"])
sketches_all_2.total_space = pd.to_numeric(sketches_all_2.total_space)
sketches_all_2.total_price = pd.to_numeric(sketches_all_2.total_price)
sketches_all_2.dtypes

sketches_all_2['mean_price'] = sketches_all_2['total_price'] / sketches_all_2['total_space']
sketches_all_2

sketches_all_3 = sketches_all_2.groupby('href2')['mean_price'].mean()
sketches_all_3

sketches_all_4 = pd.DataFrame(data=sketches_all_3)
sketches_all_4

In [ ]:
sketches_all_5 = sketches_all.copy()
sketches_all_5.total_price = sketches_all_5.total_price.str.replace('€', '')
sketches_all_5.total_price = sketches_all_5.total_price.str.replace('[+TVA]', '')
sketches_all_5.total_price = sketches_all_5.total_price.str.replace(r"[a-zA-Z]",'')
sketches_all_5.total_price = sketches_all_5.total_price.str.replace("[ț ţ ă â]", '')
sketches_all_5.total_space = sketches_all_5.total_space.str.replace(r"[a-zA-Z]",'')
sketches_all_5.total_space = sketches_all_5.total_space.str.replace("[ț ţ â ă]", '')

sketches_all_5.total_space = pd.to_numeric(sketches_all_5.total_space)
sketches_all_5.total_price = pd.to_numeric(sketches_all_5.total_price)
sketches_all_5.dtypes

sketches_all_6 = sketches_all_5.groupby('href2')['total_space'].sum()
sketches_all_6

sketches_all_7 = pd.DataFrame(data=sketches_all_6)
sketches_all_7

In [ ]:
table_all_char_sketpri_meanpr = table_all_char_sketpri.merge(sketches_all_4, how="left", on="href2")
table_all_char_sketpri_meanpr_tspace = table_all_char_sketpri_meanpr.merge(sketches_all_7, how="left", on="href2")
table_all_char_sketpri_meanpr_tspace

In [ ]:
table_all_char_sketpri_meanpr_tspace_w = table_all_char_sketpri_meanpr_tspace.merge(website_all, how="left", on="href2")
table_all_char_sketpri_meanpr_tspace_w

In [ ]:
col_dict = {'name': 'Nume',
            'address': 'Adresa',
            'developer': 'Dezvoltator',
            'city': 'Oras',
            'date': 'Data',
            'href2': 'Href',
            'main_price_from': 'Pret minim',
            'mean_price': 'Pret mediu',
            'total_space': 'Spatiu total m2',
            'website': 'Website'}   ## key→old name, value→new name

table_all_char_sketpri_meanpr_tspace_w.columns = [col_dict.get(x, x) for x in table_all_char_sketpri_meanpr_tspace_w.columns]
table_all_char_sketpri_meanpr_tspace_w

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
cols_to_order = ['Nume', 'Adresa', 'Dezvoltator', 'Oras', 'Website', 'Href', 'Pret minim', 'Pret mediu', 'Spatiu total m2']
new_columns = cols_to_order + (table_all_char_sketpri_meanpr_tspace_w.columns.drop(cols_to_order).tolist())
table_all_char_sketpri_meanpr_tspace_w = table_all_char_sketpri_meanpr_tspace_w[new_columns]
table_all_char_sketpri_meanpr_tspace_w

In [ ]:
# Export
table_all_char_sketpri_meanpr_tspace_w.to_excel("/Users/robert.radoslav/desktop/residential_final.xlsx", sheet_name="All cities")

# Colab export alternative
# from google.colab import files
# residential.to_excel('residential.xlsx') 
# files.download('residential.xlsx')

##### Old Code